# Plotting Variables and Blocks

This notebook demonstrates the built-in plotting features of pySMSpp for inspecting model data.
Both `Variable` and `Block` objects expose a `plot()` method that automatically selects an appropriate
chart type based on the data dimensionality:

| Variable shape | Default plot |
|---|---|
| scalar (0-D) | bar chart |
| 1-D array | line plot |
| 2-D array | heatmap (`kind="heatmap"`) or line plot with legend (`kind="line"`) |

In [ ]:
import numpy as np
from pysmspp import Block, Variable

## Plotting a Single Variable

### Scalar Variable

A scalar variable (0-D data) is displayed as a bar chart.

In [ ]:
scalar_var = Variable("MaxPower", "float", (), 100.0)
ax = scalar_var.plot()
ax.set_ylabel("kW")

### 1-D Variable

A 1-D variable is displayed as a line plot. The first (and only) dimension name is used as the x-axis label.

In [ ]:
time_steps = 24
demand_1d = np.abs(np.sin(np.linspace(0, 2 * np.pi, time_steps))) * 80 + 20

var_1d = Variable("ActivePowerDemand", "float", ("TimeHorizon",), demand_1d)
ax = var_1d.plot()
ax.set_ylabel("kW")

### 2-D Variable – Heatmap (default)

A 2-D variable is displayed as a heatmap by default (`kind="heatmap"`). The first dimension labels the
y-axis and the second dimension labels the x-axis.

In [ ]:
n_nodes = 3
demand_2d = np.outer(np.array([1.0, 0.8, 0.6]), demand_1d)

var_2d = Variable(
    "ActivePowerDemand",
    "float",
    ("NumberNodes", "TimeHorizon"),
    demand_2d,
)
ax = var_2d.plot()  # kind="heatmap" is the default

### 2-D Variable – Line Plot with Legend

Pass `kind="line"` to get a line plot where each row becomes a separate labelled line.

In [ ]:
ax = var_2d.plot(kind="line")
ax.set_ylabel("kW")

## Plotting All Variables in a Block

`Block.plot()` creates one subplot per variable and arranges them vertically in a single figure.
Scalar variables are skipped by default (they contain no time-series information).

In [ ]:
block = Block()
block.block_type = "UCBlock"

# 1-D variables
block.add_variable(
    "ActivePowerDemand",
    "float",
    ("TimeHorizon",),
    demand_1d,
)
block.add_variable(
    "MaxPowerFlow",
    "float",
    ("NumberLines",),
    np.array([100.0, 80.0, 60.0]),
)

# Scalar – skipped unless explicitly requested
block.add_variable("NumberUnits", "int", (), 3)

fig = block.plot()
fig

### Selecting Specific Variables

Pass a list of variable names to `variables` to control which variables are plotted.

In [ ]:
fig = block.plot(variables=["ActivePowerDemand"])
fig

### Plotting a 2-D Block Variable as Lines

Extra keyword arguments are forwarded to each `Variable.plot()` call, so you can use
`kind="line"` to switch all 2-D subplots to line-plot mode.

In [ ]:
block2d = Block()
block2d.block_type = "UCBlock"
block2d.add_variable(
    "ActivePowerDemand",
    "float",
    ("NumberNodes", "TimeHorizon"),
    demand_2d,
)

# Heatmap (default)
fig_heatmap = block2d.plot()
fig_heatmap

In [ ]:
# Line plot with per-node legend
fig_line = block2d.plot(kind="line")
fig_line